# E. Ejercicios laboratorio 2 MA4702-2021

Este laboratorio tiene 6 ejercicios.
La parte presencial consiste en los ejercicios 1 al 4 (idealmente comience los siguientes)
El resto se entrega el viernes 28 de mayo de acuerdo a instrucciones.

Observación:  
Los ejercicios 1; 2; 3a; 4b; 5a no tienen requisitos.  
Una vez tenga hecho 1, 2 y 3a puede hacer el resto de 3, 4 y 5.
Para hacer ejercicio 6 necesita todos los anteriores.lñ

Atrasos:
Cada día de atraso penalizará en 10 décimas su nota final de laboratorio
Con un máximo de 4 días de atrasos (luego de eso, tendrá un 1.0)


# 🔴 Datos grupo y Observaciones

Escriba en esta celda el nombre de cada integrante de su grupo.

- Integrante 1: Axel Alvarez V.

- Integrante 2: Benjamín Tardy D.

- Integrante 3: Diego Dominguez

- Integrante 4: Francisco Maldonado.


## Observaciones

- Si tiene alguna observación a la corrección escríbala acá.

In [1]:
#Ejecute esta celda una vez

using JuMP, Gurobi, DataFrames
const GUROBI_ENV = Gurobi.Env()

function instancia(i)
    p1 = [2,4,3]
    w1 = [5,1,3]
    A1 = [[3,1]]
    p2 = [3, 14, 4, 15, 11, 12, 12, 5, 15, 6, 13, 7, 11, 9, 5]
    w2 = [63, 41, 66, 23, 79, 23, 67, 56, 29, 74, 34, 51, 55, 42, 76]
    A2 = [[3,1],[5,7], [6,5],[10,1]]
    p3 = [9, 4, 5, 4, 12, 1, 2, 6, 1, 5, 11, 8, 2, 3, 9, 4, 11, 1, 5, 7, 8, 7, 12, 3, 2]
    w3 = [21, 8, 36, 84, 2, 3, 21, 40, 81, 92, 16, 78, 82, 49, 85, 9, 72, 35, 52, 74, 92, 36, 10, 54, 15]
    A3 = [[6, 9], [13, 17], [1, 21], [19, 24], [2, 17], [21, 22], [3, 7], [2, 25]]
    p4 = [7, 2, 2, 8, 1, 6, 1, 8, 9, 1, 2, 2, 11, 1, 11, 8, 5, 2, 8, 10, 1, 5, 5, 9, 8, 11, 3, 2, 12, 7, 7, 9]
    w4 = [95, 61, 77, 9, 35, 38, 66, 72, 84, 57, 83, 92, 35, 11, 52, 6, 92, 98, 51, 2, 72, 2, 70, 42, 38, 8, 73, 23, 20, 95, 31, 86]
    A4 = [[4, 17], [4, 26], [15, 27], [7, 32], [14, 29], [17, 22], [3, 26], [8, 13], [19, 20], [7, 16], [6, 26], [10, 27]]
    if i==1
        return p1,w1,[]
    elseif i==2
        return p1,w1,A1
    elseif i==3
        return p2,w2,[]
    elseif i==4
        return p2,w2,A2
    elseif i==5
        return p3,w3,[]
    elseif i==6
        return p3,w3,A3
    elseif i==7
        return p4,w4,[]
    elseif i==8
        return p4,w4,A4
    end
    @error "i debe estar entre 1 y 8"
    return 
end

# Ejemplo generador de precedencias
 using StatsBase
 sample([[i,j] for i in 1:20 for j in i+1:20], 8; replace=true)

Academic license - for non-commercial use only - expires 2021-07-25


8-element Array{Array{Int64,1},1}:
 [3, 7]
 [1, 14]
 [6, 15]
 [3, 14]
 [4, 16]
 [19, 20]
 [2, 18]
 [5, 10]

## 🔴 Ejercicio 1. Modelos Originales

Nota: Para cargar una instancia puede escribir
```julia
p,w,A=instancia(i)
```

Escriba funciones ```creaModeloRect1(p,w,A)``` y ```creaModeloOrden1(p,w,A)```
     que retornen modelos tipo Rect1 (como en la sección B) y tipo Orden1 (como en la sección C) respectivamente 
para instancias con tiempos de proceso p, pesos w, y precedencias A.
     En ambas funciones, su modelo debe crearse con las opciones.
     
> TimeLimit: 15  
> Threads: 1  
> Presolve: 0  
> Heuristics: 0  
> Cuts: 0  

Al final debe probar su modelo, optimizando ambos en las instancias simples i=1, i=2, i=3, i=4. 
(Nota: Es posible que para i=3 e i=4 no encuentre solución, no se preocupe, pero revise que los óptimos coincidan en las intancias 1 y 2)



In [2]:
function creaModeloRect1(p,w,A)
    # 1. Crear modelo, asignar Gurobi como el optimizador
    modelo = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "TimeLimit" => 15,
        "Threads" => 1,
        "Presolve" => 0,
        "Heuristics" => 0,
        "Cuts" => 0))
    set_optimizer(modelo, Gurobi.Optimizer)
    
    T = [i for i in 1:sum(p)]
    J = [i for i in 1:length(w)]
    
    # Variable
    @variable(modelo, 0<=Q[i in 1:length(w),t in T]<=1, Int)
 
 
    #3. Función objetivo
    @objective(modelo, Min, sum(Q[i,t]*t*w[i] for t in T, i in J))


    # Restricciones
    for i in J
        @constraint(modelo, sum(Q[i,t] for t in T) == 1)
    end
 
    for i in J
        for t in T
            if t <= p[i] -1
                @constraint(modelo, Q[i,t] == 0)
            end
        end
    end    
  
    for i in J
        for j in J
            for t in T
                for s in T
                    if i<j && (t - p[j] + 1 <= s) && (s-p[i] + 1 <= t)
                        @constraint(modelo, Q[i,s] + Q[j,t] <= 1) 
                    end
                end
            end
        end
    end
    
    for s in T
        for a in A
            @constraint(modelo, sum(Q[a[1],t] for t in [k for k in 1:s])>=sum(Q[a[2],t] for t in [k for k in 1:s]))
        end
    end
    
    return modelo
end

creaModeloRect1 (generic function with 1 method)

In [3]:
function creaModeloOrden1(p,w,A)
    modelo = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "TimeLimit" => 15,
        "Threads" => 1,
        "Presolve" => 0,
        "Heuristics" => 0,
        "Cuts" => 0))
    #set_optimizer_attributes(modelo, "OutputFlag" => 0, "TimeLimit" => 80)
    set_optimizer(modelo, Gurobi.Optimizer)
    
    N=length(p)
    J=[i for i in 1:N]
    M=sum(i for i in p)
    
    #Variables de decisión
    @variable(modelo, Z[i in J], Int)
    @variable(modelo, 0<=y[i in J,j in J]<=1)
    
    #Restricción 1
    for i in J
        @constraint(modelo,y[i,i]==1)
    end
    
    #Restricción 2
    for i in J
        for j in J
            if i<j
                @constraint(modelo,y[i,j]+y[j,i]==1)
            end
        end
    end
    
    #Restricción 3
    for i in J
        for j in J
            for k in J
                if i<j<k || i<k<j
                    @constraint(modelo, y[i,j]+y[j,k]+y[k,i]<=2)
                end
            end
        end
    end
    
    #Restricción 4
    for a in A
        @constraint(modelo, y[a[1],a[2]]==1)
    end
    
    #Restricción 5
    for i in J
        for j in J
            if i!=j
                @constraint(modelo, Z[i]+p[j]-Z[j]<=M*(1-y[i,j]))
            end
        end
    end
    
    #Restricción 6
    for i in J
        @constraint(modelo, p[i]<=Z[i]<= M)
    end
        
    #Función objetivo ( no la he escrito, es referencia)
    @objective(modelo, Min, sum(Z[i]*w[i] for i in J))

    return modelo
end

creaModeloOrden1 (generic function with 1 method)

In [4]:
# Probar en instancia 1
p, w, A = instancia(1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)

Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25


A JuMP Model
Minimization problem with:
Variables: 12
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 6 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 8 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.Interval{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 9 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: Z, y

In [5]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 124 rows, 27 columns and 263 nonzeros
Model fingerprint: 0x90b07ba0
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 43.0000000
Presolve removed 124 rows and 27 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 34 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.400000000000e+01, best bound 3.400000000000e+01, gap 0.0000%

User-callback calls 52, time in user-callback 0.00 sec


In [6]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 17 rows, 15 columns and 39 nonzeros
Model fingerprint: 0x11c24f4c
Variable types: 12 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 9 rows and 9 columns
Presolve time: 0.00s
Presolved: 8 rows, 6 columns, 24 nonzeros
Variable types: 3 continuous, 3 integer (0 binary)

Root relaxation: objective 2.300000e+01, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      23.0000000   23.00000  0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.00 seconds
Thread count was 4 (of 4 available processors)

Solution count 1

In [7]:
# Probar en instancia 2
p, w, A = instancia(2)
m1=creaModeloRect1(p,w,A);
m2=creaModeloOrden1(p,w,A);

Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25


In [8]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 133 rows, 27 columns and 353 nonzeros
Model fingerprint: 0xed99d14a
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 70.0000000
Presolve removed 133 rows and 27 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 43 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.300000000000e+01, best bound 4.300000000000e+01, gap 0.0000%

User-callback calls 58, time in user-callback 0.00 sec


In [9]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 18 rows, 15 columns and 40 nonzeros
Model fingerprint: 0xca3040f7
Variable types: 12 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 12 rows and 10 columns
Presolve time: 0.00s
Presolved: 6 rows, 5 columns, 16 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)
Found heuristic solution: objective 38.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 38 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.800000000000e+01, best bound 3.800000000000e+01, gap 0.0000%

User-callback calls 36, time in user-callback 0.00 sec


In [10]:
# Probar en instancia 3
p, w, A = instancia(3)
m1=creaModeloRect1(p,w,A);
m2=creaModeloOrden1(p,w,A);

Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25


In [11]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 257420 rows, 2130 columns and 516813 nonzeros
Model fingerprint: 0x1f4f4dd3
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 247450 rows and 127 columns
Presolve time: 2.62s
Presolved: 9970 rows, 2003 columns, 171122 nonzeros
Variable types: 0 continuous, 2003 integer (2003 binary)

Root relaxation: objective 1.567819e+04, 862 iterations, 0.15 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15678.1881    0  115          - 15678.1881      -     -    3s
     0     0 25680.5834    0  105          - 25680.5834      -     -    4s
     0   

In [12]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1255 rows, 255 columns and 3615 nonzeros
Model fingerprint: 0x96ba0820
Variable types: 240 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 12837.000000
Presolve removed 135 rows and 135 columns
Presolve time: 0.00s
Presolved: 1120 rows, 120 columns, 3360 nonzeros
Variable types: 105 continuous, 15 integer (0 binary)

Root relaxation: objective 6.642000e+03, 105 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6642.0000000 6642.00000  0.00%     -    0s

Explored 0 nodes (105 simplex iterations) in 0.0

In [13]:
# Probar en instancia 4
p, w, A = instancia(4)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)

Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25


A JuMP Model
Minimization problem with:
Variables: 240
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 124 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1120 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.Interval{Float64}`: 15 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 225 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 225 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: Z, y

In [14]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 257988 rows, 2130 columns and 598037 nonzeros
Model fingerprint: 0x946b71a2
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 247544 rows and 157 columns
Presolve time: 3.77s
Presolved: 10444 rows, 1973 columns, 238453 nonzeros
Variable types: 0 continuous, 1973 integer (1973 binary)

Root relaxation: objective 1.557801e+04, 926 iterations, 0.25 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 16208.3170    0  116          - 16208.3170      -     -    5s
     0     0 26464.4823    0  123          - 26464.4823      -     -    7s
     0  

In [15]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1259 rows, 255 columns and 3619 nonzeros
Model fingerprint: 0xc924bec1
Variable types: 240 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]
Presolve removed 220 rows and 140 columns
Presolve time: 0.00s
Presolved: 1039 rows, 115 columns, 3064 nonzeros
Variable types: 100 continuous, 15 integer (0 binary)

Root relaxation: objective 9.509000e+03, 96 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    9509.0000000 9509.00000  0.00%     -    0s

Explored 0 nodes (96 simplex iterations) in 0.01 seconds
Thread count was 4 (of 4 available proces

## 🔴 Ejercicio 2. Transformación de soluciones


2.a)
Escriba una función ```generafactible(orden, p, modelo)``` que recibe:
* Un arreglo ```orden``` que contiene una permutación del arreglo 1:N (donde N es el largo de p)
* Un arreglo ```p``` de pesos de una instancia
* Un índice ```modelo``` en {1,2}

Si ```modelo=1``` debe devolver la solución asociada al orden entregado  para Rect1 y si ```modelo=2```
debe devolver la solución asociada al orden para Rect2.

Por ejemplo, la solución asociada a p=[2,4,3] y orden=[3,2,1] debe tener tiempos de completación C(3)=3, C(2)=3+4=7, C(1)=3+4+2=9. Con esto, si ejecuta:

(Ejecute estas lineas al final de 2.a)
```julia
orden=[3,2,1];
p=[2,4,3];
QQ=generafactible(orden,p,1);
ZZ,yy=generafactible(orden,p,2);
@show QQ;
@show ZZ;
@show yy;
```
debe obtener como salida
```julia
QQ = [0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 1 0 0; 0 0 1 0 0 0 0 0 0]
ZZ = [9, 7, 3]
yy = [1 0 0; 1 1 0; 1 1 1]
```


2.b) 

En la parte anterior se solicitó una función que dado un orden entrega soluciones factibles para los modelos creados. En esta parte debe hacer el proceso inverso: escribir una función que recupere el orden de los trabajos a partir de una solución factible.

Más precisamente, escriba dos funciones ```generapermutacionRect(QQ)``` y ```generapermutacionOrden(ZZ,yy)``` que hagan lo siguiente

* Dado una matriz QQ factible para Rect1, ```generapermutacionRect(QQ)``` entrega el orden en el cual están asignados los trabajos en esta solución.

* Dado (ZZ, yy) factible para Orden1, ```generapermutacionOrden(ZZ,yy)``` entrega el orden en el cual están asignados los trabajos en esta solución.

Siguiendo el ejemplo anterior, si ahora ejecuta.

(Ejecute estas lineas al final de 2.b)
```julia
QQ = [0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 1 0 0; 0 0 1 0 0 0 0 0 0];
ZZ = [9, 7, 3];
yy = [1 0 0; 1 1 0; 1 1 1];
orden1=generapermutacionRect(QQ);
orden2=generapermutacionOrden(ZZ,yy);
@show orden1
@show orden2
```
debe obtener como salida
```
orden1 = [3, 2, 1]
orden2 = [3, 2, 1]
```


In [16]:
# 2.a)
function generafactible(orden, p, modelo)
    if modelo==1
        C=zeros(Int,length(p))
        c=0
        for i in orden
            if c==0
                C[i]=p[i]
                c=i
            else
                aux=C[c]+p[i]
                c=i
                C[i]=aux
            end
        end
        aux = maximum(C)
        I = length(p)
        QQ=zeros(Int,I,aux)
        c=1
        for j in C
            QQ[c,j]=1
            c+=1
        end
        return QQ
    elseif modelo==2
        C=zeros(Int,length(p))
        c=0
        for i in orden
            if c==0
                C[i]=p[i]
                c=i
            else
                aux=C[c]+p[i]
                c=i
                C[i]=aux
            end
        end
        ZZ=C
        yy = zeros(Int,length(p),length(p))
        for i in 1:length(orden)
            for j in 1:length(orden)
                if orden[i]<=orden[j]
                    yy[i,j]=1
                end
            end
        end
        return ZZ,yy
    end
    @error("modelo debe ser 1 o 2")
end

generafactible (generic function with 1 method)

In [17]:
# prueba 2.a
orden=[3,2,1];
p=[2,4,3];
QQ=generafactible(orden,p,1);
ZZ,yy=generafactible(orden,p,2);
@show QQ;
@show ZZ;
@show yy;

QQ = [0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 1 0 0; 0 0 1 0 0 0 0 0 0]
ZZ = [9, 7, 3]
yy = [1 0 0; 1 1 0; 1 1 1]


In [18]:
# 2.b)
function generapermutacionRect(QQ)
    # CUIDADO! QQ podria tener valores muy cercanos a 1, pero no ser iguales a 1 por problemas de precisión
    # Si necesita revisar que QQ[i,t] es 1, use QQ[i,t]≈1
    C=zeros(Int,size(QQ,1))
    for i in 1:size(QQ,1)
        for t in 1:size(QQ,2)
            if QQ[i,t]≈1
                C[i]=t
            end
        end
    end
    orden=zeros(Int,length(C))
    for i in 1:length(C)
        c=0
        v=C[i]
        for j in 1:length(C)
           if v>=C[j]
                c+=1
            end
        end
        orden[c]=i
    end
    return orden
end
function generapermutacionOrden(ZZ,yy)
    # CUIDADO! yy podria tener valores muy cercanos a 1, pero no ser iguales a 1 por problemas de precisión
    # Si necesita revisar que yy[i,j] es 1, use yy[i,t]≈1
    orden=zeros(Int,length(ZZ))
    for i in 1:size(yy,1)
        ant=-1
        for j in 1:size(yy,2)
            if yy[i,j]≈1
                ant+=1
            end
        end
        orden[length(ZZ)-ant]=i
    end
    return orden
end
    

generapermutacionOrden (generic function with 1 method)

In [19]:
# prueba 2.b
QQ = [0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 1 0 0; 0 0 1 0 0 0 0 0 0];
ZZ = [9, 7, 3];
yy = [1 0 0; 1 1 0; 1 1 1];
orden1=generapermutacionRect(QQ);
orden2=generapermutacionOrden(ZZ,yy);
@show orden1
@show orden2

orden1 = [3, 2, 1]
orden2 = [3, 2, 1]


3-element Array{Int64,1}:
 3
 2
 1

## 🔴 Ejercicio 3. Generando buenas soluciones iniciales para ambos modelos.

3.a)
Sabemos que si no hay restricciones de precedencias, el orden dado por mayor a menor eficiencia es una solución óptima para el problema. Este orden se puede obtener en julia a partir de w y p escribiendo
```julia
eficiencia = sort(1:length(p), by=x->w[x]/p[x], rev=true)
```
Es posible que este orden no sea compatible con las restricciones de precedencias dadas por el digrafo (J,A) en ciertas instancias por lo que no puede ser usado como solucion inicial directamente.

Una alternativa simple para obtener un orden inicial ```inicial``` bueno es la siguiente.

Para cada elemento i, determine el conjunto prec[i] de sus predecesores, es decir el conjunto de todos los elementos j tal que [i,j] está en A. Partiendo del arreglo ```inicial``` vacío y hasta que este arreglo contenga todos los elementos de J, determine iterativamente cual es el primer elemento del arreglo ```eficiencia``` tal que todos sus predecesores están en ```inicial```, una vez encontrado dicho elemento, agréguelo al final del arreglo  ```inicial``` y elimínelo de  ```eficiencia```.

Haga todo lo anterior dentro de una función  ```ordeneficiente(p,w,A)``` que entregue el arreglo ```inicial``` descrito arriba. Luego, pruebe su función con los arreglos de la instancia 4, en particular si escribe
```julia
p,w,A=instancia(4)
inicial=ordeneficiente(p,w,A)
@show inicial
```
debería obtener
```julia
inicial = [3, 15, 10, 1, 8, 12, 13, 14, 2, 11, 9, 6, 5, 7, 4]
```

3.b)
Usando las funciones creadas en 3.a y 2.a. 

Cree y optimice ambos modelos en las instancias i=1, i=2, i=3, i=4 pero ahora usando como solución inicial la solución factible asociada al orden obtenido mediante ```orden_eficiente(p,w,A)```. Llamamos a esto solucion inicial eficiente

3.c) Pregunta abierta: ¿Cómo compara (gap, calidad, tiempo, etc.) las soluciones encontradas en 2.c, con las soluciones encontradas en el ejercicio 1?


In [20]:
# 3.a)
# Hint. algunos comandos utiles para manejar arreglos son push!, deleteat!, 

function ordeneficiente(p, w, A)
    eficiencia = sort(1:length(p), by=x->w[x]/p[x], rev=true)
    N=length(p)
    prec=[[] for i=1:N]
    
    # Para todo i, llenar en prec[i] el conjunto de todos los predecesores de i
    for i in 1:N
        for j in 1:N
            a=[j,i]
            if a in A
                push!(prec[a[2]],a[1])
            end
        end
    end
                    
    inicial = Int[]
    while(!isempty(eficiencia))
         # encontrar primer indice i tal que todas las precedencias de eficiencia[i] están en inicial
         for i in eficiencia 
            guardar=true
            for j in prec[i]
                if !(j in inicial)
                    guardar=false
                end
            end
   
         # al encontrarlo, agregar eficiencia[i] al final de inicial y eliminarlo  de eficiencia 
             if guardar
                push!(inicial,i)
                setdiff!(eficiencia,[i])
                break
            end
         end
    end
    return inicial
end

ordeneficiente (generic function with 1 method)

In [21]:
# 3.a) prueba
p,w,A=instancia(4)
inicial=ordeneficiente(p,w,A)
@show inicial;

inicial = [3, 15, 10, 1, 8, 12, 13, 14, 2, 11, 9, 6, 5, 7, 4]


In [22]:
# 3.b) Probar ambos modelos en instancia 1

p, w, A = instancia(1)
inicial=ordeneficiente(p,w,A)

# crear modelos (Rect1) y modelo2 (Orden1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)
# obtener solucion inicial QQ a partir de inicial
QQ=generafactible(inicial,p,1);
ZZ,yy=generafactible(inicial,p,2);

# fijar QQ como solucion inicial de m1
for i in size(QQ,1)
    for j in size(QQ,2)
        set_start_value(m1[:Q][i,j],QQ[i,j])
    end
end

# fijar (ZZ,yy) como solucion inicial de m2
for i in size(ZZ,1)
    for j in size(ZZ,2)
        set_start_value(m2[:Z][i,j],ZZ[i,j])
    end
end

for i in size(yy,1)
    for j in size(yy,2)
        set_start_value(m2[:y][i,j],yy[i,j])
    end
end

Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25


In [23]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 124 rows, 27 columns and 263 nonzeros
Model fingerprint: 0xe4685d6a
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 34 (0.00s)
Loaded user MIP start with objective 34

Presolve removed 124 rows and 27 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 34 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.400000000000e+01, best bound 3.400000000000e+01, gap 0.0000%

User-callback calls 48, time in user-callback 0.00 sec


In [24]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 17 rows, 15 columns and 39 nonzeros
Model fingerprint: 0x28be108f
Variable types: 12 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 7e+00]

User MIP start produced solution with objective 29 (0.00s)
Loaded user MIP start with objective 29

Presolve removed 9 rows and 9 columns
Presolve time: 0.00s
Presolved: 8 rows, 6 columns, 24 nonzeros
Variable types: 3 continuous, 3 integer (0 binary)

Root relaxation: objective 2.300000e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      23.0000000   23.00000  0.00%     -    0s

Explored 0 nodes (1

In [25]:
# 3.b) Probar ambos modelos en instancia 2

p, w, A = instancia(2)
inicial=ordeneficiente(p,w,A)

# crear modelos (Rect1) y modelo2 (Orden1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)
# obtener solucion inicial QQ a partir de inicial
QQ=generafactible(inicial,p,1);
ZZ,yy=generafactible(inicial,p,2);

# fijar QQ como solucion inicial de m1
for i in size(QQ,1)
    for j in size(QQ,2)
        set_start_value(m1[:Q][i,j],QQ[i,j])
    end
end

# fijar (ZZ,yy) como solucion inicial de m2
for i in size(ZZ,1)
    for j in size(ZZ,2)
        set_start_value(m2[:Z][i,j],ZZ[i,j])
    end
end

for i in size(yy,1)
    for j in size(yy,2)
        set_start_value(m2[:y][i,j],yy[i,j])
    end
end

Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25


In [26]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 133 rows, 27 columns and 353 nonzeros
Model fingerprint: 0xba3b4c98
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 43 (0.00s)
Loaded user MIP start with objective 43

Presolve removed 133 rows and 27 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 43 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.300000000000e+01, best bound 4.300000000000e+01, gap 0.0000%

User-callback calls 54, time in user-callback 0.00 sec


In [27]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 18 rows, 15 columns and 40 nonzeros
Model fingerprint: 0x1214d69c
Variable types: 12 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 7e+00]

User MIP start produced solution with objective 38 (0.00s)
Loaded user MIP start with objective 38

Presolve removed 12 rows and 10 columns
Presolve time: 0.00s
Presolved: 6 rows, 5 columns, 16 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 38 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.800000000000e+01, best bound 3.800000000000e+01, gap 0.0000%

User-callback calls 40, time in user-callback 0.00 sec


In [28]:
# 3.b) Probar ambos modelos en instancia 3

p, w, A = instancia(3)
inicial=ordeneficiente(p,w,A)

# crear modelos (Rect1) y modelo2 (Orden1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)
# obtener solucion inicial QQ a partir de inicial
QQ=generafactible(inicial,p,1);
ZZ,yy=generafactible(inicial,p,2);

# fijar QQ como solucion inicial de m1
for i in size(QQ,1)
    for j in size(QQ,2)
        set_start_value(m1[:Q][i,j],QQ[i,j])
    end
end

# fijar (ZZ,yy) como solucion inicial de m2
for i in size(ZZ,1)
    for j in size(ZZ,2)
        set_start_value(m2[:Z][i,j],ZZ[i,j])
    end
end

for i in size(yy,1)
    for j in size(yy,2)
        set_start_value(m2[:y][i,j],yy[i,j])
    end
end

Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25


In [29]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 257420 rows, 2130 columns and 516813 nonzeros
Model fingerprint: 0x4dde0d4a
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 35578 (6.51s)
Loaded user MIP start with objective 35578
Processed MIP start in 6.53 seconds

Presolve removed 247450 rows and 127 columns
Presolve time: 1.92s
Presolved: 9970 rows, 2003 columns, 171122 nonzeros
Variable types: 0 continuous, 2003 integer (2003 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+01   0.000000e+00      8s
     862    1.5678188e+04   0.000000e+00   0.000000e+00      9s

Root relaxation: objecti

In [30]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1255 rows, 255 columns and 3615 nonzeros
Model fingerprint: 0x71f4c632
Variable types: 240 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]

User MIP start produced solution with objective 7174 (0.00s)
Loaded user MIP start with objective 7174

Presolve removed 135 rows and 135 columns
Presolve time: 0.00s
Presolved: 1120 rows, 120 columns, 3360 nonzeros
Variable types: 105 continuous, 15 integer (0 binary)

Root relaxation: objective 6.642000e+03, 105 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6642.0000000 6642.00000  0.00%     -

In [31]:
# 3.b) Probar ambos modelos en instancia 4

p, w, A = instancia(4)
inicial=ordeneficiente(p,w,A)

# crear modelos (Rect1) y modelo2 (Orden1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)
# obtener solucion inicial QQ a partir de inicial
QQ=generafactible(inicial,p,1);
ZZ,yy=generafactible(inicial,p,2);

for i in size(QQ,1)
    for j in size(QQ,2)
        set_start_value(m1[:Q][i,j],QQ[i,j])
    end
end

for i in size(ZZ,1)
    for j in size(ZZ,2)
        set_start_value(m2[:Z][i,j],ZZ[i,j])
    end
end

for i in size(yy,1)
    for j in size(yy,2)
        set_start_value(m2[:y][i,j],yy[i,j])
    end
end

Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25


In [32]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 257988 rows, 2130 columns and 598037 nonzeros
Model fingerprint: 0xec7cb814
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 38017 (9.40s)
Loaded user MIP start with objective 38017
Processed MIP start in 17.92 seconds

Presolve removed 247544 rows and 157 columns
Presolve time: 2.89s
Presolved: 10444 rows, 1973 columns, 238453 nonzeros
Variable types: 0 continuous, 1973 integer (1973 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+01   0.000000e+00     21s
     926    1.5578012e+04   0.000000e+00   0.000000e+00     21s

Root relaxation: objec

In [33]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1259 rows, 255 columns and 3619 nonzeros
Model fingerprint: 0x812c7b3c
Variable types: 240 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]

User MIP start produced solution with objective 9813 (0.00s)
Loaded user MIP start with objective 9813

Presolve removed 220 rows and 140 columns
Presolve time: 0.00s
Presolved: 1039 rows, 115 columns, 3064 nonzeros
Variable types: 100 continuous, 15 integer (0 binary)

Root relaxation: objective 9.509000e+03, 85 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    9509.0000000 9509.00000  0.00%     - 

 🔴 3.c)
 
 ¿Cómo compara (tiempo, calidad, etc.) las soluciones encontradas en 3.b, con las soluciones encontradas en el ejercicio 1?
 
Los valores objetivos en el ejercicio 1 y 3 coinciden, excepto al ejecutar la instancia 3 en el modelo 2, en donde el valor objetivo obtenido en el ejercicio 1 (6,64) es menor al del obtenido en el ejercicio 3 (9,5). Además la cantidad de iteraciones realizadas en el ejercicio 3 (105) también es mayor que la cantidad de iteraciones realizadas en el ejercicio 1 (67). Teniendo en cuenta que lo que se busca es minimizar el valor de la función objetivo, el ejercicio 1 muestra una mayor calidad que el ejercicio 3 en esta instancia y modelo en donde los resultados obtenidos son distintos.  

En el resto de instancias y modelos, ambos ejercicios entregan los mismos valores objetivos. En general el número de iteraciones realizados por el ejercicio 1 es menor, habiendo contadas excepciones (instancia 1 modelo 1 e instancia 1 modelo 2) y habiendo una gran diferencia en las instancias 3 y 4, donde la diferencia de iteraciones sonbrepasa las 2.000. 

En conclusión, asumiendo que la mejor calidad es la que ofrece el ejercicio que ofrece un equilibrio entre entregar valores objetivos menores (pues en este caso buscamos minimizar) y hacerlo en el menor número de iteraciones, se concluye que lo realizado en el ejercicio 1 ofrece una mayor calidad que el procedimiento llevado a cabo en el ejercicio 3.
 
 
 

## 🔴 Ejercicio 4. Mejorando Rect1 con CliqueCuts

4.a) 

Nuevamente cree un modelo Rect1 para las instancias i=3, i=4, al igual que la parte anterior
use la solucion inicial eficiente. Antes de optimizar, agregue generación agresiva de CliqueCuts y resuelva.

Luego responda la pregunta abierta: ¿Cómo compara las soluciones encontradas en 4.a, con las soluciones encontradas en 3.b?

4.b)

De acuerdo a lo explicado en la sección D.3, podemos obtener un mejor modelo para nuestro problema generando nosotros mismos los CliqueCuts. Esto ese logra reemplazando en el modelo (rect1) las restricciones

$$Q_{i,s}+Q_{j,t} \leq 1,\quad \forall i< j, \forall s,t\in [T]: (t-p_j+1\leq s) \wedge (s-p_i+1\leq t)$$

por las restricciones

$$\sum_{(i,s)\in I(t)} Q_{i,s} \leq 1, \quad \forall t\in T$$
donde $I(t) = \{(i,s)\colon i\in J, s\in [T], s-p_i+1\leq t\leq s\}$

Llamemos Rect2 al modelo resultante.

En base a esto, escriba una función ```creaModeloRect2(p,w,A)``` que retorne un modelo tipo Rect2 para el problema, usando las mismas opciones que en la pregunta 1.a)

4.c) 
Use este modelo para resolver las instancias i=3, i=4 (no genere CliqueCuts) y use solución inicial eficiente. 
Pregunta abierta: ¿Cómo compara las soluciones (tiempo, calidad, etc.) encontradas en 4.c con las de 4.a?

In [34]:
# 4.a) i=3. Cliquecuts en Rect1
p, w, A = instancia(3)
inicial=ordeneficiente(p,w,A)
m=creaModeloRect1(p,w,A)
QQ=generafactible(inicial,p,1);
# fijar QQ como solucion inicial de m
for i in size(QQ,1)
    for j in size(QQ,2)
        set_start_value(m[:Q][i,j],QQ[i,j])
    end
end
# agregar CliqueCuts
set_optimizer_attribute(m,"CliqueCuts",2)
# optimizar
optimize!(m)

Academic license - for non-commercial use only - expires 2021-07-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 257420 rows, 2130 columns and 516813 nonzeros
Model fingerprint: 0x4dde0d4a
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 35578 (6.53s)
Loaded user MIP start with objective 35578
Processed MIP start in 6.56 seconds

Presolve removed 247450 rows and 127 columns
Presolve time: 1.92s
Presolved: 9970 rows, 2003 columns, 171122 nonzeros
Variable types: 0 continuous, 2003 integer (2003 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+01   0.000000e+00      9s
     862    1.5678188

In [35]:
# 4.a) i=4. Cliquecuts en Rect1
p, w, A = instancia(4)
inicial=ordeneficiente(p,w,A)
m=creaModeloRect1(p,w,A)
QQ=generafactible(inicial,p,1);
# fijar QQ como solucion inicial de m
for i in size(QQ,1)
    for j in size(QQ,2)
        set_start_value(m[:Q][i,j],QQ[i,j])
    end
end
# agregar CliqueCuts
set_optimizer_attribute(m,"CliqueCuts",2)
# optimizar
optimize!(m)

Academic license - for non-commercial use only - expires 2021-07-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 257988 rows, 2130 columns and 598037 nonzeros
Model fingerprint: 0xec7cb814
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 38017 (9.19s)
Loaded user MIP start with objective 38017
Processed MIP start in 17.81 seconds

Presolve removed 247544 rows and 157 columns
Presolve time: 2.99s
Presolved: 10444 rows, 1973 columns, 238453 nonzeros
Variable types: 0 continuous, 1973 integer (1973 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+01   0.000000e+00     21s
     926    1.55780

4.a) 🔴 Pregunta abierta. 

¿Cómo compara las soluciones encontradas en 4.a, con las soluciones encontradas en 3.b?

En lo que refiere a la instancia 3, las soluciones entregadas por ambos métodos fueron relativamente cercanas, siendo 3,8 la respuesta entregada por el procedimiento utilizado en 3.b y 3,55 la entregada por el de 4 (ambas respuestas están multiplicadas por 10.000, por eso lo de "relativamente" cercanas). Con lo anterior, hay que agregar que el número de iteraciones realizado en 4.a (3.845) es poco más de la mitad de las realizada en 3.b (7.697).   
Por lo anterior, y el criterio de que la calidad va acompañada con un menor (o mayor, en caso de que se busque maximizar) resultado y un menor número de iteraciones (lo que acompaña a un menor tiempo de ejecución) podemos concluir que la función utilizada en 4.a cuenta con una calidad superior a la de la función utilizada en el 3.b.  



In [36]:
# 4.b)
function creaModeloRect2(p,w,A)
    modelo = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "TimeLimit" => 15,
        "Threads" => 1,
        "Presolve" => 0,
        "Heuristics" => 0,
        "Cuts" => 0))
    set_optimizer(modelo, Gurobi.Optimizer)
    
    T = [i for i in 1:sum(p)]
    J = [i for i in 1:length(w)]
    
    # 2. Declarar variables de decisión
    @variable(modelo, 0<=Q[i in 1:length(w),t in T]<=1, Int)
    
    #3. Función objetivo
    @objective(modelo, Min, sum(Q[i,t]*t*w[i] for t in T, i in J))
    
    # Restricciones
    for i in J
        @constraint(modelo, sum(Q[i,t] for t in T) == 1)
    end
    for i in J
        for t in T
            if t <= p[i] -1
                @constraint(modelo, Q[i,t] == 0)
            end
        end
    end    
    
    for t in T
        @constraint(modelo, sum(Q[i,s] for i in J, s in T if s-p[i]+1 <= t <= s )<=1)
    end
    
    
    for s in T
        for a in A
            @constraint(modelo, sum(Q[a[1],t] for t in [k for k in 1:s])>=sum(Q[a[2],t] for t in [k for k in 1:s]))
        end
    end
    return modelo
end


creaModeloRect2 (generic function with 1 method)

In [37]:
# 4.c) i=3 usar modelo Rect2, con solucion inicial.
p, w, A = instancia(3)
inicial=ordeneficiente(p,w,A)
m=creaModeloRect2(p,w,A)
QQ=generafactible(inicial,p,1);
# fijar QQ como solucion inicial de m
for i in size(QQ,1)
    for j in size(QQ,2)
        set_start_value(m[:Q][i,j],QQ[i,j])
    end
end

# optimizar
optimize!(m)

Academic license - for non-commercial use only - expires 2021-07-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 284 rows, 2130 columns and 21699 nonzeros
Model fingerprint: 0x8d6abb9f
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 35578 (0.05s)
Loaded user MIP start with objective 35578

Presolve removed 131 rows and 159 columns
Presolve time: 0.05s
Presolved: 153 rows, 1971 columns, 20325 nonzeros
Variable types: 0 continuous, 1971 integer (1971 binary)

Root relaxation: cutoff, 684 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

In [38]:
# 4.c) i=4 usar modelo Rect2, con solucion inicial.
p, w, A = instancia(4)
inicial=ordeneficiente(p,w,A)
m=creaModeloRect2(p,w,A)
QQ=generafactible(inicial,p,1);
# fijar QQ como solucion inicial de m
for i in size(QQ,1)
    for j in size(QQ,2)
        set_start_value(m[:Q][i,j],QQ[i,j])
    end
end

# optimizar
optimize!(m)

Academic license - for non-commercial use only - expires 2021-07-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 852 rows, 2130 columns and 102923 nonzeros
Model fingerprint: 0xc74a897d
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 38018 (0.23s)
User MIP start produced solution with objective 38017 (0.53s)
Loaded user MIP start with objective 38017

Sparsify removed 67977 nonzeros (83%)
Presolve removed 211 rows and 7 columns
Presolve time: 0.14s
Presolved: 641 rows, 2123 columns, 14218 nonzeros
Variable types: 0 continuous, 2123 integer (1968 binary)

Root relaxation: objective 3.748759e+04, 964 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objec

4.c) Pregunta abierta: ¿Cómo compara las soluciones (tiempo, calidad, etc.) encontradas en 4.c con las de 4.a?

Los valores objetivos obtenidos por 4.c igualan a los obtenidos en 4.a, sin embargo el número de iteraciones bajó significativamente. Pasando de 3845 a 105 en la instancia 3 y de 7029 a 405 en la instancia 4. Tomando en cuenta que el valor objetivo entregado en ambas partes fue el mismo y que el número de iteraciones bajó significativamente entre ambas partes, la conclusión es que el método utilizado en la parte 4.c es el que presenta mejor calidad.

## 🔴 Ejercicio 5. Eliminando big-M en modelo Orden1


El problema central del modelo Orden1 es la existencia de restricciones de tipo big-M.

$$Z_i+p_j-Z_j\leq M(1-y_{i,j}), \forall i, j\in J, i\neq j$$

Una forma interesante de cambiar estas restricciones es notar que el tiempo de completación se puede calcular directamente a partir de las variables $y$ como sigue

$$Z_i = \sum_{j\in J}y_{j,i}p_j$$

Con esto automáticamente sabemos que si $y_{i,j}$ es 1 entonces $Z_i+p_i\leq Z_j$.

Llamemos entonces Orden2 al modelo obtenido reemplazando las restricciones
$$Z_i+p_j-Z_j\leq M(1-y_{i,j}), \forall i, j\in J, i\neq j$$
por las restricciones
$$Z_i = \sum_{j\in J}y_{j,i}p_j, \forall i\in J$$


5.a) 
    Escriba una función ```creaModeloOrden2(p,w,A)``` que retorne un modelo tipo Orden2 para el problema (use las mismas opciones que en la pregunta 1.a) 

5.b) 
    Use este modelo para resolver las instancias i=3, i=4 partiendo de la solución inicial eficiente.

5.c) Pregunta abierta: ¿Cómo compara las soluciones (tiempo, calidad, etc.) asociadas al modelo Orden2 con las del modelo Orden1 resueltas hasta ahora?


In [39]:
# 5.a)
function creaModeloOrden2(p,w,A)
    modelo = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "TimeLimit" => 15,
        "Threads" => 1,
        "Presolve" => 0,
        "Heuristics" => 0,
        "Cuts" => 0))
    set_optimizer(modelo, Gurobi.Optimizer)
    
    N=length(p)
    J=[i for i in 1:N]
    M=sum(i for i in p)
    
    #Variables de decisión
    @variable(modelo, Z[i in J], Int)
    @variable(modelo, 0<=y[i in J,j in J]<=1)
    
    #Restricción 1
    for i in J
        @constraint(modelo,y[i,i]==1)
    end
    
    #Restricción 2
    for i in J
        for j in J
            if i<j
                @constraint(modelo,y[i,j]+y[j,i]==1)
            end
        end
    end
    
    #Restricción 3
    for i in J
        for j in J
            for k in J
                if i<j<k || i<k<j
                    @constraint(modelo, y[i,j]+y[j,k]+y[k,i]<=2)
                end
            end
        end
    end
    
    #Restricción 4
    for a in A
        @constraint(modelo, y[a[1],a[2]]==1)
    end
    
    # [NUEVA] Restricción 5
    for i in J
        @constraint(modelo, Z[i]==sum(y[j,i]*p[j] for j in J))
    end
    
    #Restricción 6
    for i in J
        @constraint(modelo, p[i]<=Z[i]<= M)
    end
        
    #Función objetivo ( no la he escrito, es referencia)
    @objective(modelo, Min, sum(Z[i]*w[i] for i in J))

    return modelo
end

creaModeloOrden2 (generic function with 1 method)

In [40]:
# 5.b) i=3 usar modelo Orden2, con solucion inicial.
p, w, A = instancia(3)
inicial=ordeneficiente(p,w,A)
m2=creaModeloOrden2(p,w,A);
# obtener solucion inicial QQ a partir de inicial
ZZ,yy=generafactible(inicial,p,2);

for i in size(ZZ,1)
    for j in size(ZZ,2)
        set_start_value(m2[:Z][i,j],ZZ[i,j])
    end
end

for i in size(yy,1)
    for j in size(yy,2)
        set_start_value(m2[:y][i,j],yy[i,j])
    end
end
optimize!(m2)

Academic license - for non-commercial use only - expires 2021-07-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1060 rows, 255 columns and 3225 nonzeros
Model fingerprint: 0xa6e468d8
Variable types: 240 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 2e+00]

User MIP start produced solution with objective 35578 (0.02s)
Loaded user MIP start with objective 35578

Presolve removed 135 rows and 135 columns
Presolve time: 0.00s
Presolved: 925 rows, 120 columns, 2955 nonzeros
Variable types: 105 continuous, 15 integer (0 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 35578 

Optimal solution found (tolerance 1.00

In [41]:
# 5.b) i=4 usar modelo Orden2, con solucion inicial.
p, w, A = instancia(4)
inicial=ordeneficiente(p,w,A)
m2=creaModeloOrden2(p,w,A);
# obtener solucion inicial QQ a partir de inicial
ZZ,yy=generafactible(inicial,p,2);

for i in size(ZZ,1)
    for j in size(ZZ,2)
        set_start_value(m2[:Z][i,j],ZZ[i,j])
    end
end

for i in size(yy,1)
    for j in size(yy,2)
        set_start_value(m2[:y][i,j],yy[i,j])
    end
end
optimize!(m2)

Academic license - for non-commercial use only - expires 2021-07-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1064 rows, 255 columns and 3229 nonzeros
Model fingerprint: 0xa103b727
Variable types: 240 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 2e+00]

User MIP start produced solution with objective 38018 (0.02s)
Loaded user MIP start with objective 38018

Presolve removed 215 rows and 140 columns
Presolve time: 0.00s
Presolved: 849 rows, 115 columns, 2669 nonzeros
Variable types: 100 continuous, 15 integer (0 binary)

Root relaxation: objective 3.801700e+04, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*

5.c) Pregunta abierta: ¿Cómo compara las soluciones (tiempo, calidad, etc.) asociadas al modelo Orden2 con las del modelo Orden1 resueltas hasta ahora (3b)?

Resultados obtenidos:

Instancia 3:
 Orden1:
  - Valor Objetivo:  6.64e+3
  - Iteraciones: 49
 Orden2: 
  - Valor objetivo: 3.55e+04
  - Iteraciones: 42

Instancia 4:
 Orden1:
  - Valor Objetivo:  9.50e+3
  - Iteraciones: 54
 Orden2: 
  - Valor objetivo: 3.80e+04
  - Iteraciones: 55
  
Como se puede observar, los resultados obtenidos en ambas instancias para el modelo Orden2 entrega peores soluciones en comparación al modelo Orden1. Esto se puede deber a un error en la implementación en las restricciones, lo que provoca que la mejora implementada en el nuevo modelo de orden no sean significativas. De todas formas, ambos modelos tardan un tiempo similar en encontrar una solución, por lo que ambos son igual de eficientes en tiempo. 
 
 



## 🔴 Ejercicio 6. Analisis completo


En este problema deberá comparar las 4 soluciones siguientes 


* Modelo1: Rect1, *con* CliqueCuts, con solución inicial eficiente.
* Modelo2: Rect2,  con solución inicial eficiente.
* Modelo3: Orden1, con solución inicial eficiente.
* Modelo4: Orden2, con solución inicial eficiente.

en las instancias 1 a 8 provistas, con el mismo tiempo límite (15 segundos).

Más precisamente cree una función

```Julia
function Benchmark()
return df
```

donde df es un `dataframe` cuyas filas están indexadas por instancia, y las columnas son
    
| tiempo1 | gap1 | valor1 | tiempo2 | gap2 | valor2 | tiempo3 | gap3 | valor3 | tiempo4 | gap4 | valor4 | ganador | mejororden |

con
tiempo j = tiempo usado en la resolución de la instancia, usando formulacion Modeloj, 
           usando MOI.get(modelo, MOI.SolveTime())
           
gap j    = gap de resolución en porcentaje (0% = optimo, infinito si no encontró solución factible)  
           usando MOI.get(modelo, MOI.RelativeGap())
           
valor j  = valor objetivo encontrado  

ganador   = mejor modelo de los 4 para esta instancia. Esto se calcula de la siguiente manera:  
           1) Revisar si algun modelo encontró el óptimo revisando si  
              termination_status(model) == MOI.OPTIMAL  
           2) Si uno o mas encontraron el optimo, el mejor es aquel con menor tiempo de resolución.  
           3) Si ninguno encuentra el optimo, aquel con menor gap.  

mejororden = orden de los trabajos para el modelo ganador en esta instancia (encontrado usando las funciones de las partes anteriores).  

Al finalizar, ejecute la función e imprima en pantalla el dataframe con
```julia
df=Benchmark()
ENV["COLUMNS"]=1000
df
```

**Advertencia:** La ejecución de la función podría tomar hasta 10 minutos. Planee con tiempo.

In [42]:
function Benchmark()
    df = DataFrame(
        tiempo1 = Float64[],
        gap1 = Float64[],
        valor1            =  Float64[],
        tiempo2           =  Float64[],
        gap2              =  Float64[],
        valor2            =  Float64[],
        tiempo3           =  Float64[],
        gap3              =  Float64[],
        valor3            =  Float64[],
        tiempo4           =  Float64[],
        gap4              =  Float64[],
        valor4            =  Float64[],
        ganador           =  Int64[],
        mejororden        =  Any[])
    
    for i=1:8
        # Crear instancia y solucion inicial
        p,w,A= instancia(i)
        
        m1=creaModeloRect1(p,w,A)
        m2=creaModeloRect2(p,w,A)
        m3=creaModeloOrden1(p,w,A)
        m4=creaModeloOrden2(p,w,A)
    
        # Resolver cada uno y extraer parametros pedidos (puede usar 
        #MOI.get(m1, MOI.SolveTime()), MOI.get(m1, MOI.RelativeGap())
    
        optimize!(m1)
        tiempo1=MOI.get(m1, MOI.SolveTime())
        gap1=MOI.get(m1, MOI.RelativeGap())
        valor1=objective_value(m1)

        optimize!(m2)
        tiempo2=MOI.get(m2, MOI.SolveTime())
        gap2=MOI.get(m2, MOI.RelativeGap())
        valor2=objective_value(m2)

        optimize!(m3)
        tiempo3=MOI.get(m3, MOI.SolveTime())
        gap3=MOI.get(m3, MOI.RelativeGap())
        valor3=objective_value(m3)

        optimize!(m4)
        tiempo4=MOI.get(m4, MOI.SolveTime())
        gap4=MOI.get(m4, MOI.RelativeGap())
        valor4=objective_value(m4)

        # Determinar ganador (de acuerdo a GAP) y mejor ordedn
        ganador = argmin([gap1,gap2,gap3,gap4])
        modelo_ganador=[m1,m2,m3,m4][ganador]
        if ganador<3
            mejororden= generapermutacionRect(JuMP.value.(modelo_ganador[:Q]))
        else
            mejororden= generapermutacionOrden(JuMP.value.(modelo_ganador[:Z]),JuMP.value.(modelo_ganador[:y]))
        end

        # agregar fila a dataframe
        push!(df, [tiempo1, gap1, valor1, tiempo2, gap2, valor2, tiempo3, gap3,valor3 ,tiempo4 ,gap4, valor4, ganador, mejororden])
    end
     return df
end

Benchmark (generic function with 1 method)

In [43]:
df = Benchmark()
ENV["COLUMNS"]=1000
df

Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25
Academic license - for non-commercial use only - expires 2021-07-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 124 rows, 27 columns and 263 nonzeros
Model fingerprint: 0x90b07ba0
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 43.0000000
Presolve removed 124 rows and 27 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 34 

Optimal solution fo

,tiempo1,gap1,valor1,tiempo2,gap2,valor2,tiempo3,gap3,valor3,tiempo4,gap4,valor4,ganador,mejororden
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Any
1,0.0,0.0,34.0,0.0,0.0,34.0,0.0,0.0,23.0,0.0,0.0,34.0,1,"[1, 3, 2]"
2,0.0,0.0,43.0,0.0,0.0,43.0,0.0,0.0,38.0,0.0,0.0,43.0,1,"[3, 1, 2]"
3,10.5781,0.0,35578.0,0.0624847,0.0,35578.0,0.0156479,0.0,6642.0,0.0,0.0,35578.0,1,"[1, 3, 15, 10, 8, 12, 5, 7, 13, 14, 2, 11, 9, 6, 4]"
4,21.8845,0.0,38017.0,0.620312,0.0,38017.0,0.00895119,0.0,9509.0,0.0069828,0.0,38017.0,1,"[3, 10, 1, 15, 8, 12, 13, 6, 5, 7, 14, 2, 11, 9, 4]"
5,21.4767,0.0,43562.0,0.0624847,0.0,43562.0,0.0312157,0.0,6404.0,0.0156498,0.0,43562.0,1,"[9, 13, 18, 4, 10, 24, 14, 21, 20, 7, 19, 12, 15, 25, 3, 8, 17, 22, 6, 1, 16, 2, 11, 23, 5]"
6,49.9748,0.0,47045.0,2.56189,0.0,47045.0,0.0312443,0.0,8648.0,0.031271,0.0,47045.0,1,"[6, 9, 13, 18, 4, 10, 14, 19, 24, 20, 12, 15, 3, 7, 8, 1, 21, 2, 25, 17, 22, 16, 11, 23, 5]"
7,31.9578,4.34109e-5,69107.0,0.124969,0.0,69107.0,0.0468674,0.0,8544.0,0.0468369,0.0,69107.0,2,"[21, 7, 10, 18, 12, 11, 3, 5, 2, 27, 17, 23, 30, 1, 28, 14, 32, 9, 8, 19, 6, 25, 15, 24, 31, 13, 29, 4, 16, 26, 22, 20]"
8,92.055,0.0,76573.0,3.62415,0.0,76573.0,0.0625153,0.0,10581.0,0.0468636,0.0,76573.0,1,"[21, 7, 10, 18, 12, 11, 3, 5, 2, 23, 1, 30, 28, 14, 32, 9, 8, 15, 27, 4, 17, 19, 6, 25, 24, 31, 13, 29, 16, 26, 22, 20]"
